In [7]:
%matplotlib inline
import sys
BIN = ''
sys.path.append(BIN)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

import my_matplotlib_style as ms
import matplotlib as mpl
mpl.rc_file(BIN + 'my_matplotlib_rcparams')

from fastai import data_block, basic_train, basic_data
import fastai

In [8]:
from nn_utils import AE_basic, AE_bn, AE_LeakyReLU, AE_bn_LeakyReLU

In [32]:
# Load data
#train = pd.read_pickle(BIN + 'processed_data/aod/partial_train.pkl')
#test = pd.read_pickle(BIN + 'processed_data/aod/partial_test.pkl')

#train = pd.read_pickle(BIN + 'processed_data/aod/full_train.pkl')
#test = pd.read_pickle(BIN + 'processed_data/aod/full_test.pkl')

train = pd.read_pickle(BIN + 'processed_data/aod/all_jets_partial_train_10percent.pkl')
test = pd.read_pickle(BIN + 'processed_data/aod/all_jets_partial_test_10percent.pkl')

#train = pd.read_pickle(BIN + 'processed_data/aod/all_jets_full_train.pkl')
#test = pd.read_pickle(BIN + 'processed_data/aod/all_jets_full_test.pkl')

#train['pt'] = train['pt'] / 1000.  # Convert to GeV
#test['pt'] = test['pt'] / 1000.  # Convert to GeV
#train['m'] = train['m'] / 1000.  # Convert to GeV
#test['m'] = test['m'] / 1000.  # Convert to GeV
#train['NegativeE'] = train['NegativeE'] / 1000.  # Convert to GeV
#test['NegativeE'] = test['NegativeE'] / 1000.  # Convert to GeV

#train['LeadingClusterPt'] = train['LeadingClusterPt'] / 1000.  # Convert to GeV
#test['LeadingClusterPt'] = test['LeadingClusterPt'] / 1000.  # Convert to GeV

#train['LeadingClusterSecondR'] = train['LeadingClusterSecondR'] / 1000.  # Convert to GeV
#test['LeadingClusterSecondR'] = test['LeadingClusterSecondR'] / 1000.  # Convert to GeV

#train['LeadingClusterSecondLambda'] = train['LeadingClusterSecondLambda'] / 1000.  # Convert to GeV
#test['LeadingClusterSecondLambda'] = test['LeadingClusterSecondLambda'] / 1000.  # Convert to GeV

train.pop('JetGhostArea')
test.pop('JetGhostArea')
train.pop('BchCorrCell')
test.pop('BchCorrCell')

# Remove all jets with EMFrac outside (-2, 2)
train = train[(train['EMFrac'] < 2) & (train['EMFrac'] > -2)]
test = test[(test['EMFrac'] < 2) & (test['EMFrac'] > -2)]

FileNotFoundError: [Errno 2] No such file or directory: 'processed_data/aod/all_jets_partial_train_10percent.pkl'

In [33]:
for key in test.keys():
    print(key)

NameError: name 'test' is not defined

In [34]:
len(train.keys())

NameError: name 'train' is not defined

In [12]:
train.head()

NameError: name 'train' is not defined

In [13]:
def plot_data(data, e_range=None, pt_range=None, save=False, title=True, logy=False):

    plt.figure()
    plt.hist(data['pt'], bins=100, range=pt_range)
    plt.xlabel(r'$p_T$')
    if title:
        plt.suptitle('Train set')
    plt.ylabel('Number of jets')
    ms.sciy()
    if logy:
        plt.yscale('log')
    if save:
        plt.savefig('data_exploration_figures/hist_pt_train.png')
    
    plt.figure()
    plt.hist(data['eta'], bins=100)
    plt.xlabel(r'$\eta$')
    if title:
        plt.suptitle('Train set')
    plt.ylabel('Number of jets')
    ms.sciy()
    if save:
        plt.savefig('data_exploration_figures/hist_eta_train.png')
    
    plt.figure()
    plt.hist(data['phi'], bins=100)
    plt.xlabel(r'$\phi$')
    if title:
        plt.suptitle('Train set')
    plt.ylabel('Number of jets')
    ms.sciy()
    if save:
        plt.savefig('data_exploration_figures/hist_phi_train.png')    
        
    plt.figure()
    plt.hist(data['m'], bins=100, range=e_range)
    plt.xlabel(r'$m$')
    if title:
        plt.suptitle('Train set')
    plt.ylabel('Number of jets')
    ms.sciy()
    if logy:
        plt.yscale('log')
    if save:
        plt.savefig('data_exploration_figures/hist_m_train.png')

In [14]:
plot_data(test, logy=True)

NameError: name 'test' is not defined

In [15]:
test.keys()

NameError: name 'test' is not defined

In [16]:
def plot_all(data, logy=False):
    for key in data.keys():
        plt.figure()
        plt.hist(data[key], bins=200)
        plt.xlabel(str(key))
        plt.ylabel('Number of jets')
        if logy:
            plt.yscale('log')
        else:
            ms.sciy()

In [17]:
logy = True
plot_all(train, logy)

NameError: name 'train' is not defined

In [18]:
# Normalize
train_mean = train.mean()
train_std = train.std()

normed_train = (train - train_mean) / train_std
normed_test = (test - train_mean) / train_std

NameError: name 'train' is not defined

In [19]:
train_std

NameError: name 'train_std' is not defined

In [20]:
plot_all(data=normed_test, logy=True)

NameError: name 'normed_test' is not defined

In [21]:
keys = ['pt',
        'eta',
        'phi',
        'm',
        'ActiveArea',
        'ActiveArea4vec_eta',
        'ActiveArea4vec_m',
        'ActiveArea4vec_phi',
        'ActiveArea4vec_pt',
        'AverageLArQF',
        'HECQuality',
        'LArQuality',
        'Width',
        'WidthPhi',
        'CentroidR',
        'DetectorEta',
        'LeadingClusterCenterLambda',
        'LeadingClusterPt',
        'LeadingClusterSecondLambda',
        'LeadingClusterSecondR',
        'N90Constituents',
        'EMFrac',
        'Timing',
        'OotFracClusters10',
        'HECQuality',
        'OotFracClusters5']
lognormed_variables = ['pt',
                       'LeadingClusterPt',
                       'LeadingClusterCenterLambda',
                       'LeadingClusterSecondLambda',
                       'LeadingClusterSecondR',
                       'CentroidR',
                       'LArQuality',                      
                      ]
custom_variables = [
                    'DetectorEta',
                    'ActiveArea4vec_eta',
                    'EMFrac',
                    'NegativeE',
                    ]
notnormed_variables = ['ActiveArea',
                       'ActiveArea4vec_m',
                       'ActiveArea4vec_pt',
                       'HECFrac',
                       'OotFracClusters10',
                       'OotFracClusters5',
                      ]

In [22]:
custom_normalized_test = test.copy()
for var in lognormed_variables:
    custom_normalized_test[var] = np.log10(test[var] + 100) - 2
custom_normalized_test['DetectorEta'] = test['DetectorEta'] / 5
custom_normalized_test['ActiveArea4vec_eta'] = test['ActiveArea4vec_eta'] / 5
custom_normalized_test['EMFrac'] = test['EMFrac'] / 5
custom_normalized_test['NegativeE'] = np.log10(-test['NegativeE'] + 1) / 4

NameError: name 'test' is not defined

In [23]:
for var in lognormed_variables + custom_variables:
    plt.figure()
    plt.hist(custom_normalized_test[var], bins=50)
    plt.xlabel(var)
    plt.yscale('log')

NameError: name 'custom_normalized_test' is not defined

<Figure size 1200x900 with 0 Axes>

In [24]:
from FunctionScaler import FunctionScaler
fs = FunctionScaler.FunctionScaler(FunctionScaler.TransformedFunction_Uniform(-1,1), downplay_outofbounds_lower_n_range=None, downplay_outofbounds_upper_n_range=None, downplay_outofbounds_lower_set_point=None, downplay_outofbounds_upper_set_point= None) # calling the normal function by name
#fs = FunctionScaler.FunctionScaler("gauss-11")
#fs = FunctionScaler.FunctionScaler("gauss")

ModuleNotFoundError: No module named 'FunctionScaler'

In [25]:
scaling_data = test.values
fs.fit(scaling_data[:len(scaling_data)//5])

NameError: name 'test' is not defined

In [26]:
scaled_data = fs.transform(scaling_data)

NameError: name 'fs' is not defined

In [27]:
scaled_df = pd.DataFrame(scaled_data, columns=test.columns)
scaled_df

NameError: name 'scaled_data' is not defined

In [28]:
plot_all(scaled_df)

NameError: name 'scaled_df' is not defined

In [29]:
unscaled_data = fs.invtransform(scaled_data)
unscaled_df = pd.DataFrame(unscaled_data, columns=test.columns)
unscaled_df

NameError: name 'fs' is not defined

In [30]:
plt.figure()
plt.hist(unscaled_df['pt'], bins=100, color='b')
plt.hist(train['pt'], bins=100, color='r', alpha=0.5)

NameError: name 'unscaled_df' is not defined

<Figure size 1200x900 with 0 Axes>

In [31]:
plot_all(unscaled_df)

NameError: name 'unscaled_df' is not defined